In [2]:
import numpy as np
import pandas as pd
import xlrd
import openpyxl

import os
folder_name = '成品仓库进出明细/'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

month_name = '成品仓库进出明细'
input_filename = month_name +'.xls'
output_filename = folder_name + month_name+ '_添加类别.xlsx'

df = pd.read_excel(input_filename, sheetname ='成品仓库进出明细')
df.columns.values


array(['单据', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '属性',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', '入库统计', 'Unnamed: 18',
       '出库统计', 'Unnamed: 20'], dtype=object)

In [3]:
df.drop(['入库统计','Unnamed: 18'], axis=1, inplace = True)

names = df.loc[0,:]
print(names[names == '类别名称'].index)
df.rename(index=str, columns={'Unnamed: 8': 'type_name'},inplace=True)
print(df.type_name.unique())

df.rename(index=str, columns={'Unnamed: 9': 'pd_size'}, inplace = True)

df.insert(8,'type',df.type_name)
df.columns.values

Index(['Unnamed: 8'], dtype='object')
['类别名称' 'XG' 'W' 'XF' '花片' '腰线' 'HNG' 'WM' 'HNJ' 'HDF' 'XW' 'PG' '文具' 'XT'
 '浮雕' '欧驰' 'HNT' 'HNF' 'HDJ' 'HDT' '其它费用' 'HY' 'HM' 'HTF' '纸箱' '古宝斯']


array(['单据', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '属性',
       'Unnamed: 6', 'Unnamed: 7', 'type', 'type_name', 'pd_size',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', '出库统计', 'Unnamed: 20'], dtype=object)

In [4]:
df.type[df.type_name == '类别名称'] = '类别'

df.type[(df.type_name =='HDF') | (df.type_name == 'HDG') | (df.type_name == 'HDJ') | (df.type == 'HDT') ] = '地砖'
df.type[df.type_name =='HM'] = '木纹砖'
df.type[(df.type_name =='HNF') | (df.type_name == 'HNG') | (df.type_name == 'HNJ') | (df.type == 'HNT') ] = '瓷片'
df.type[df.type_name =='HTF'] = '地砖'
df.type[df.type_name == 'HW'] = '微晶'
df.type[df.type_name == 'HY'] = '全抛釉'

df.type[df.type_name == 'PG'] ='通体砖'
df.type[(df.type_name == 'PG') & (df.pd_size == '300X300')] = '地砖'
df.type[(df.type_name == 'PG') & (df.pd_size == '300X600')] = '瓷片'

df.type[(df.type_name == 'P') | (df.type_name == 'T')] = '通体砖'
df.type[(df.type_name == 'W') | (df.type_name == 'WM')] = '瓷片'
df.type[df.type_name == 'XF'] = '地砖'
df.type[(df.type_name == 'XG') | (df.type_name == 'XT') | (df.type_name == 'XW')] = '瓷片'
df.type[df.type_name == 'H3S'] = '色晶砖'
df.type[df.type_name == 'HS'] = '大理石'
df.type[df.type_name == '浮雕'] = '浮雕'
df.type[df.type_name == '文具'] = '五金仓'
df.type[df.type_name == '花片'] = '花片'
df.type[df.type_name == '欧驰'] = '欧驰'
df.type[df.type_name == '腰线'] = '腰线'
df.type[df.type_name == '纸箱'] = '纸箱'

print(df.groupby('type').type.agg('count'))

type
五金仓      111
全抛釉        4
其它费用       1
古宝斯        4
地砖       243
木纹砖        3
欧驰        13
浮雕        28
瓷片      1557
类别         1
纸箱         2
腰线        50
花片        76
通体砖       12
Name: type, dtype: int64


In [5]:
#modified output
output_df = df.copy(deep=True)
output_df.columns = output_df.iloc[0,:]
output_df.drop(['0'], axis=0,inplace = True)
output_df.set_index('编号',inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
output_df.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory

In [6]:
## regroup data
df2 = output_df.copy(deep=True)
df2['销售类型'].unique()

array(['折价销售', '正常销售', '免费样板', '收费样板', '物资自用', '免费广告', '销售费用'], dtype=object)

In [7]:
# 销售收入 : '正常销售 + 折价销售 + 收费样板 + 销售调价

#sales = df2[ (df2['销售类型'] == '正常销售') | (df2['销售类型'] == '折价销售') | (df2['销售类型'] == '收费样板') | (df2['销售类型'] == '销售调价') ]
sales = df2[ (df2['销售类型'] != '挂账样板') & (df2['销售类型'] != '挂账广告') & (df2['销售类型'] != '免费广告') & (df2['销售类型'] != '免费样板') ]


# 挂账样板

model = df2[ df2['销售类型'] == '挂账样板' ]

# 挂账广告

adv = df2[ df2['销售类型'] == '挂账广告']

# 免费广告

free_adv = df2[ df2['销售类型'] == '免费广告']

# 免费样板

free_model = df2[ df2['销售类型'] == '免费样板']


In [8]:
# 不同销售类型的金额汇总

output_filename = folder_name + '不同销售类型金额汇总.xlsx'
group = df2.groupby('销售类型')['金额'].agg(np.sum)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
group.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory


In [9]:
df2.columns.values

array(['日期', '类型', '销售类型', '往来单位', '仓库名称', '产品编号', '产品名称', '类别', '类别名称',
       '规格', '庄数', '色号', '等级', '品牌', '包装', '单位', '单价', '数量(片)', '金额'], dtype=object)

In [10]:
#销售收入

df_temp = sales 
output_filename = folder_name +'summary_销售收入.xlsx'
output_filename2 = folder_name +'往来单位_销售收入.xlsx'


group = pd.concat([ df_temp.groupby('类别')['数量(片)'].agg(np.sum),
                    df_temp.groupby('类别')['金额'].agg(np.sum)],
                   axis = 1)

group.loc[group.shape[0]] = group.sum()
group.rename(index = {group.shape[0]-1: '总计'}, inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
group.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory


group2 = df_temp.groupby('往来单位')['金额'].agg(np.sum).reset_index().set_index('往来单位')

group2.loc[group2.shape[0]] = group2.sum()
group2.rename(index = {group2.shape[0]-1: '总计'}, inplace = True)

writer = pd.ExcelWriter(output_filename2) #creates instance of an excel workboo
group2.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory


In [11]:
#挂账样板

df_temp = model 
output_filename = folder_name +'/summary_挂账样板.xlsx'
output_filename2 = folder_name +'/往来单位_挂账样板.xlsx'


group = pd.concat([ df_temp.groupby('类别')['数量(片)'].agg(np.sum),
                    df_temp.groupby('类别')['金额'].agg(np.sum)],
                   axis = 1)

group.loc[group.shape[0]] = group.sum()
group.rename(index = {group.shape[0]-1: '总计'}, inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
group.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory

group2 = df_temp.groupby('往来单位')['金额'].agg(np.sum).reset_index().set_index('往来单位')

group2.loc[group2.shape[0]] = group2.sum()
group2.rename(index = {group2.shape[0]-1: '总计'}, inplace = True)

writer = pd.ExcelWriter(output_filename2) #creates instance of an excel workboo
group2.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory


In [12]:
#免费样板

df_temp = free_model
output_filename = folder_name +'/summary_免费样板.xlsx'
output_filename2 = folder_name +'/往来单位_免费样板.xlsx'


group = pd.concat([ df_temp.groupby('类别')['数量(片)'].agg(np.sum),
                    df_temp.groupby('类别')['金额'].agg(np.sum)],
                   axis = 1)

group.loc[group.shape[0]] = group.sum()
group.rename(index = {group.shape[0]-1: '总计'}, inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
group.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory

group2 = df_temp.groupby('往来单位')['金额'].agg(np.sum).reset_index().set_index('往来单位')

group2.loc[group2.shape[0]] = group2.sum()
group2.rename(index = {group2.shape[0]-1: '总计'}, inplace = True)

writer = pd.ExcelWriter(output_filename2) #creates instance of an excel workboo
group2.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory


In [13]:
#免费广告

df_temp = free_adv
output_filename = folder_name +'/summary_免费广告.xlsx'
output_filename2 = folder_name +'/往来单位_免费广告.xlsx'


group = pd.concat([ df_temp.groupby('类别')['数量(片)'].agg(np.sum),
                    df_temp.groupby('类别')['金额'].agg(np.sum)],
                   axis = 1)

group.loc[group.shape[0]] = group.sum()
group.rename(index = {group.shape[0]-1: '总计'}, inplace = True)

writer = pd.ExcelWriter(output_filename) #creates instance of an excel workboo
group.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory


group2 = df_temp.groupby('往来单位')['金额'].agg(np.sum).reset_index().set_index('往来单位')

group2.loc[group2.shape[0]] = group2.sum()
group2.rename(index = {group2.shape[0]-1: '总计'}, inplace = True)

writer = pd.ExcelWriter(output_filename2) #creates instance of an excel workboo
group2.to_excel(writer,'sheet1') #creates workbook 
writer.save() #saves workbook to file in python file directory
